In [1]:
import cx_Oracle
import pandas as pd

conn =  cx_Oracle.connect(user="pc",password="p201404",dsn="siatchdesa")
query = "select periodo,customer_id,product_id,clase_escalada clase from L_VW_DATOS_TRAIN_MINMAX" 
df = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM

# Para concatenar todos los chunks en un solo DataFrame (si tienes suficiente RAM)
df_full = pd.concat(df, ignore_index=True)
conn.close()
print(df_full.shape)
# Imprimir las primeras filas del DataFrame completo
print(df_full.head())
# Imprimir los tipos de datos de las columnas del DataFrame completo
print(df_full.dtypes)
# Imprimir el número de filas y columnas del DataFrame completo
print(f"Número de filas: {df_full.shape[0]}, Número de columnas: {df_full.shape[1]}")

C:\Users\pcablinski.RWD\AppData\Local\Temp\ipykernel_191576\2351707876.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM


(10143267, 4)
   PERIODO  CUSTOMER_ID  PRODUCT_ID     CLASE
0   201701        10348       20654  0.002670
1   201701        10223       20654  0.008021
2   201701        10392       20654  0.000000
3   201701        10082       20654  0.000000
4   201701        10416       20654  0.003346
PERIODO          int64
CUSTOMER_ID      int64
PRODUCT_ID       int64
CLASE          float64
dtype: object
Número de filas: 10143267, Número de columnas: 4


In [6]:
# Usando este código como base para un modelo de regresión con LightGBM
# agregar hojas de LightGBM

import lightgbm as lgb

# Variables predictoras y objetivo
X = df_full[['PERIODO', 'CUSTOMER_ID', 'PRODUCT_ID']]
y = df_full['CLASE']

# Crear el dataset de LightGBM
train_data = lgb.Dataset(X, label=y)

# Definir parámetros para regresión
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
}

# Entrenar el modelo
model = lgb.train(params, train_data, num_boost_round=100)

print("Modelo de regresión entrenado con todo el dataset.")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.056554 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 545
[LightGBM] [Info] Number of data points in the train set: 10143267, number of used features: 3
[LightGBM] [Info] Start training from score 0.004893
Modelo de regresión entrenado con todo el dataset.


In [7]:
# Cargo los datos sobre los que quiero hacer predicciones

conn =  cx_Oracle.connect(user="pc",password="p201404",dsn="siatchdesa")
query = "select * from L_DATOS_PREDICCION" 
df_pred = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM

# Para concatenar todos los chunks en un solo DataFrame (si tienes suficiente RAM)
df_pred_full = pd.concat(df_pred, ignore_index=True)
conn.close()

C:\Users\pcablinski.RWD\AppData\Local\Temp\ipykernel_191576\3968896330.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pred = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM


In [8]:
# Con el modelo entrenado, hacemos predicciones 
X_pred = df_pred_full[['PERIODO', 'CUSTOMER_ID', 'PRODUCT_ID']]
predictions = model.predict(X_pred)
# Agregar las predicciones al DataFrame original
df_pred_full['PREDICCIONES'] = predictions
# Imprimir las primeras filas del DataFrame con las predicciones
print(df_pred_full.head())
# Guardar el DataFrame con las predicciones en un archivo CSV
df_pred_full.to_csv('predicciones.csv', index=False)
# Imprimir el número de filas y columnas del DataFrame con las predicciones
print(f"Número de filas: {df_pred_full.shape[0]}, Número de columnas: {df_pred_full.shape[1]} con predicciones.")
# Guardar el modelo entrenado
model.save_model('modelo_regresion.txt')


   PERIODO  PRODUCT_ID  CUSTOMER_ID  TN_PREDICCION  PREDICCIONES
0   201912       20003        10001       0.323924      0.233927
1   201912       20027        10001       0.182536      0.187748
2   201912       20059        10001       0.284770      0.192685
3   201912       20063        10001       0.275874      0.192685
4   201912       20074        10001       0.199980      0.185543
Número de filas: 333840, Número de columnas: 5 con predicciones.


In [9]:
# Con el DataFrame de predicción, actualizamos la base de datos
# el criterio es actualizar la tabla L_DATOS_PREDICCION con las nuevas predicciones
# la columnna PREDICCIONES se debe actualizar con los nuevos valores
# la clave primaria es (PERIODO, CUSTOMER_ID, PRODUCT_ID)
# Hacer commit cada 10000 filas para evitar problemas de memoria

# Conectar a la base de datos para actualizar los datos de predicción
conn = cx_Oracle.connect(user="pc", password="p201404", dsn="siatchdesa")
# Crear un cursor para ejecutar las actualizaciones
cursor = conn.cursor()
# Iterar sobre las filas del DataFrame con las predicciones
for index, row in df_pred_full.iterrows():
    periodo = row['PERIODO']
    customer_id = row['CUSTOMER_ID']
    product_id = row['PRODUCT_ID']
    prediccion = row['PREDICCIONES']
    
    # Actualizar la tabla L_DATOS_PREDICCION con la nueva predicción
    update_query = """
        UPDATE L_DATOS_PREDICCION
        SET TN_PREDICCION = :prediccion
        WHERE PERIODO = :periodo AND CUSTOMER_ID = :customer_id AND PRODUCT_ID = :product_id
    """
    cursor.execute(update_query, {'prediccion': prediccion, 'periodo': periodo, 'customer_id': customer_id, 'product_id': product_id})  
    # Hacer commit cada 10000 filas para evitar problemas de memoria
    if index % 10000 == 0:
        conn.commit()
        print(f"Actualizadas {index} filas de L_DATOS_PREDICCION con las nuevas predicciones.")
# Confirmar los cambios en la base de datos
conn.commit()
# Cerrar el cursor y la conexión
cursor.close()
conn.close()
# Imprimir mensaje de finalización
print("Actualización de la tabla L_DATOS_PREDICCION completada con las nuevas predicciones.")


Actualizadas 0 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 10000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 20000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 30000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 40000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 50000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 60000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 70000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 80000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 90000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 100000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 110000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 120000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 